In [1]:
import sys
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.metrics import classification_report

In [2]:
df = pd.read_pickle('semicon_wafer_final.pkl')

In [3]:
df.head()

,wafer map,die size,lot number,wafer index,failure type,clean wafer,failure type int,interarea_intercubic,interarea_interlinear,interarea_intercubic_clean,interarea_interlinear_clean
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,15,none,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",7,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,16,none,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",7,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,20,Loc,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,4,none,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",7,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,12,Edge-Loc,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [4]:
X = df['interarea_intercubic']
y = df['failure type int']
X = np.stack(X)
#y = np.stack(y)
X = np.expand_dims(X, axis=-1)
#y = np.expand_dims(y, axis=-1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
X_train.shape

(16238, 40, 40, 1)

In [6]:
y.dtype

dtype('int64')

In [7]:
X.shape

(24237, 40, 40, 1)

In [8]:
X_valid, X_train = X_train[:2000], X_train[2000:]
y_valid, y_train = y_train[:2000], y_train[2000:]

In [9]:
df1 = df.groupby('failure type int', as_index = False)['lot number'].count()

In [10]:
class_names = list(df1['failure type int'])

In [11]:
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                        input_shape = [40, 40, 1]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(8, activation="softmax")
])

In [16]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer='Adam',
             metrics=['accuracy'])

In [17]:
history = model.fit(X_train, y_train, epochs=40,
                   validation_data=(X_valid, y_valid))

Epoch 1/40
445/445 [==============================] - 71s 158ms/step - loss: 1.9206 - accuracy: 0.2293 - val_loss: 1.4275 - val_accuracy: 0.5555
Epoch 2/40
445/445 [==============================] - 70s 157ms/step - loss: 1.1906 - accuracy: 0.5613 - val_loss: 0.7754 - val_accuracy: 0.7170
Epoch 3/40
445/445 [==============================] - 68s 154ms/step - loss: 0.9107 - accuracy: 0.6631 - val_loss: 0.6682 - val_accuracy: 0.7495
Epoch 4/40
445/445 [==============================] - 69s 154ms/step - loss: 0.7819 - accuracy: 0.7216 - val_loss: 0.6760 - val_accuracy: 0.7670
Epoch 5/40
445/445 [==============================] - 72s 163ms/step - loss: 0.6718 - accuracy: 0.7620 - val_loss: 0.5604 - val_accuracy: 0.7960
Epoch 6/40
445/445 [==============================] - 69s 156ms/step - loss: 0.6321 - accuracy: 0.7820 - val_loss: 0.4760 - val_accuracy: 0.8270
Epoch 7/40
445/445 [==============================] - 70s 158ms/step - loss: 0.5745 - accuracy: 0.7999 - val_loss: 0.5301 - val_ac

In [ ]:
model.evaluate(x=X_test, y=y_test)

In [ ]:
model.evaluate(x=X_train, y=y_train)

In [ ]:
model.evaluate(x=X_valid, y=y_valid)

In [ ]:
y_pred = model.predict_classes(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
model.save('model_interarea_intercubic_clean.h5')